# Function Optimisation and torchbearer Metrics

We've already seen how to train a simple CNN with torchbearer, lets now have a look at something slightly different, optimising a simple function. 

Recall the tutorial Jon gave on Autograd recently ([here](https://github.com/ecs-vlc/understanding-autograd)), we shall be optimising the function in the final section [here](https://render.githubusercontent.com/view/ipynb?commit=e9da73fbe13b709989bcd066492e1058bae0aab2&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f6563732d766c632f756e6465727374616e64696e672d6175746f677261642f653964613733666265313362373039393839626364303636343932653130353862616530616162322f5079546f72636841442e6970796e62&nwo=ecs-vlc%2Funderstanding-autograd&path=PyTorchAD.ipynb&repository_id=141979756&repository_type=Repository#Gradient-descent-&-gradients-with-respect-to-a-vector).

This function is: 

$f(x) = x_1^2 + x_2^2 + x_3^2$

with expected minimum at $x = (0,0,0)$.

## Structure

To do this we will need four things:
- a pytorch model that represents the function to be optimised
- an optimser bound to the model to update it
- a measure of loss
- a torchbearer metric to view progress

## Model

Lets start with the model. 
We put $x$ as a member variable for the model and tell PyTorch that we want to optimise it by setting it as a [Parameter](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html?highlight=parameter). 
Can you implement the forward pass to return $f(x)$?

In [1]:
import torch
from torch.nn import Module

import torchbearer as tb

class Net(Module):
    def __init__(self, x):
        super().__init__()
        self.x = torch.nn.Parameter(x)

    def f(self):
        # TODO: Implement this
        pass
        
    def forward(self, _):
        return self.f()
    
p = torch.tensor([2.0, 1.0, 10.0])
model = Net(p)

## Optimiser

For this example, we shall use SGD to optimise:

In [2]:
optim = torch.optim.SGD(model.parameters(), lr=0.0005)

## Loss

We have a very simple loss for this problem, the value of the function, since this is what we want to minimise (and is non-negative). As in pytorch, torchbearer losses are a function of y_pred and y_true, the predictions of the network and the targets respectively. In this example we do not have targets so we can ignore that variable. 

In [3]:
def loss(y_pred, y_true):
    return y_pred

## Torchbearer Metrics

Metrics form the main output of a torchbearer trial. They are logged when logging callbacks are used and they are displayed when printer callbacks are used.

Below is an example of a simple metric that inherits the base metric class which accepts a name on init, and outputs on process the current x value, which it grabs from the pytorch model instance. Metrics must output a dict so they can be shown and logged, for this a decorator is provided that just wraps the output of process with a dict. 

In [4]:
@tb.metrics.to_dict
class est(tb.metrics.Metric):
    def __init__(self):
        super().__init__('x') # Named 'x'

    def process(self, state):
        return state[tb.MODEL].x.data # Value = x

## Training

We can now easily perform optimisation for 10000 steps whilst printing the current estimate for the minimum. Note that since we have no data in this case, we use [for_training_steps](https://torchbearer.readthedocs.io/en/latest/code/main.html#torchbearer.trial.Trial.for_steps) instead of providing the trial a generator. 



In [5]:
training_steps = 10000
tbtrial = tb.Trial(model, optim, loss, metrics=[est(), 'loss']).for_train_steps(training_steps).to('cpu')
_ = tbtrial.run()

0/1(t): 100%|██████████| 10000/10000 [00:17<00:00, 574.01it/s, est=tensor([0.0001, 0.0000, 0.0005], device='cuda:0'), running_loss=2.3e-07, loss=5.25, loss_std=15.8]


This should show the function converging to the correct minimum.

## Exercise: 

Implement the function to optimise

## Moving on

The next notebook is the VAE notebook where we'll run a VAE and look a bit more at constructing callbacks, for example to visualise results. 